# Reproducing Benji's Work

In [1]:
library(tidyverse)
options(repr.plot.width=2, repr.plot.height=3)

workdir <- "/media/clint/Data/GitRepo/Statistics/BIOS706/Midterm"

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
dat_frmgham <- read_csv(file.path(workdir, "frmgham2.csv"))
head(dat_frmgham, 3)

Parsed with column specification:
cols(
  .default = col_integer(),
  SYSBP = col_double(),
  DIABP = col_double(),
  BMI = col_double()
)
See spec(...) for full column specifications.


RANDID,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,⋯,CVD,HYPERTEN,TIMEAP,TIMEMI,TIMEMIFC,TIMECHD,TIMESTRK,TIMECVD,TIMEDTH,TIMEHYP
2448,1,195,39,106,70,0,0,26.97,0,⋯,1,0,8766,6438,6438,6438,8766,6438,8766,8766
2448,1,209,52,121,66,0,0,NA,0,⋯,1,0,8766,6438,6438,6438,8766,6438,8766,8766
6238,2,250,46,121,81,0,0,28.73,0,⋯,0,0,8766,8766,8766,8766,8766,8766,8766,8766


In [3]:
dat <- dat_frmgham
print("Before preprocess")
print(dim(dat))

dat$SEX_LAB      <- c("Men", "Women")[dat$SEX]
dat$DIABETES_LAB <- c("Not a diabetic", "Diabetic")[dat$DIABETES + 1]
dat$CURSMOKE_LAB <- c("Not current smoker", "Current smoker")[dat$CURSMOKE + 1]
dat$PERIOD_LAB   <- c("Period 1", "Period 2", "Period 3")[dat$PERIOD]
dat$AGE_LAB      <- cut(dat$AGE, c(30, 60, 80))

# add the heart rate label (Benji)
dat$HEARTRTE_LAB <- cut(dat$HEARTRTE, c(0, 100, 300), labels = c("<60-100", ">100"))

dat              <- dat %>% filter(!is.na(HEARTRTE))
dat_prep         <- dat

print("After preprocess")
print(dim(dat_prep))

[1] "Before preprocess"
[1] 11627    39
[1] "After preprocess"
[1] 11621    45


In [4]:
dat <- dat_prep
dat_per01 <- dat %>% filter(PERIOD == 1)
dat_per02 <- dat %>% filter(PERIOD == 2)

In [5]:
dat <- dat_prep
dat <- dat %>% filter(!(DIABETES == 1 & PERIOD == 1)) %>% filter(PERIOD != 3)
id  <- dat %>% group_by(RANDID) %>% summarize(Count = n()) %>% filter(Count == 2) %>% .$RANDID

dat <- dat %>% filter(RANDID %in% id)
grp <- dat %>% group_by(RANDID) %>% summarize(Group = sum(DIABETES))

dat_grp <- left_join(grp, dat, by = "RANDID")
dat_grp$Group_LAB <- c("Group ", "")

In [6]:
grp %>% group_by(Group) %>% summarise(Count = n())

Group,Count
0,3774
1,67


In [7]:
tmp <- dat_grp %>% filter(PERIOD == 1)
table(tmp$HEARTRTE_LAB, tmp$Group)

         
             0    1
  <60-100 3685   64
  >100      89    3

In [11]:
tmp <- dat_grp %>% filter(PERIOD == 1)
tab <- table(tmp$HEARTRTE_LAB, tmp$Group)
res <- chisq.test(tab)

Warning message in chisq.test(tab):
“Chi-squared approximation may be incorrect”

In [12]:
res$expected

,0,1
<60-100,3683.60479,65.39521
>100,90.39521,1.60479


In [13]:
res


	Pearson's Chi-squared test with Yates' continuity correction

data:  tab
X-squared = 0.52072, df = 1, p-value = 0.4705
